In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast
from collections import defaultdict

## **Load processed data from GitHub**

In [95]:
queries_df = pd.read_csv("https://raw.githubusercontent.com/Ron-DS-AI/Information_Retrieval/refs/heads/main/qrels/queries_rnd3_stacked.csv")
judgments_df = pd.read_csv("https://raw.githubusercontent.com/Ron-DS-AI/Information_Retrieval/refs/heads/main/qrels/qrels.csv")
corpus_df = pd.read_csv("https://raw.githubusercontent.com/Ron-DS-AI/Information_Retrieval/refs/heads/main/Preprocessing/preprocessed_metadata_with_summaries.csv")

In [99]:
corpus_df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,referenced_by_count,JournalName_DOI,tags,TagCount,summarised_abstracts
0,iua8c4hy,300ff0def92740f7e1142036f9e6ae30845e13d2,medrxiv,covid-19 outbreak in oman: model-driven impact...,10.1101/2020.04.02.20050666,NaN,NaN,medrxiv,motivated by the rapid spread of covid-19 all ...,2020-04-06,...,NaN,document_parses/pdf_json/300ff0def92740f7e1142...,NaN,https://doi.org/10.1101/2020.04.02.20050666,215782459.0,10,NaN,"['methodologies or experimental designs', 'the...",4,motivated by the rapid spread of covid-19 all ...
1,xph08fwv,9a3e8c974bc68c29b0273ee771553046d7f37930,medrxiv,automated and semi-automated contact tracing: ...,10.1101/2020.04.14.20063636,NaN,NaN,medrxiv,introduction traditional approaches to case-fi...,2020-04-17,...,NaN,document_parses/pdf_json/9a3e8c974bc68c29b0273...,NaN,http://medrxiv.org/cgi/content/short/2020.04.1...,216055053.0,2,NaN,"['literature review', 'methodologies or experi...",4,introduction traditional approaches to case-fi...
2,aosmo568,ca6949e3e039a0e4098644b0b2df90606ae3a7ee,medrxiv,efficacy of remdesivir versus placebo for the ...,10.1101/2020.04.09.20059196,NaN,NaN,medrxiv,background: in spite of the global containment...,2020-04-14,...,NaN,document_parses/pdf_json/ca6949e3e039a0e409864...,NaN,http://medrxiv.org/cgi/content/short/2020.04.0...,215782112.0,1,NaN,"['literature review', 'methodologies or experi...",4,background: in spite of the global containment...
3,ujomta30,d831dbf38025a44e0436fd11c52af4db9eb7c5b0,medrxiv,hydroxychloroquine versus covid-19: a rapid sy...,10.1101/2020.04.14.20065276,NaN,NaN,medrxiv,background: coronavirus disease 2019 (covid-19...,2020-04-20,...,NaN,document_parses/pdf_json/d831dbf38025a44e0436f...,NaN,http://medrxiv.org/cgi/content/short/2020.04.1...,216035656.0,15,NaN,"['literature review', 'methodologies or experi...",4,background: coronavirus disease 2019 (covid-19...
4,xwlzq3m3,44eec5c0f84d7069033431dcb5a6adeb01b8ee0c,elsevier; medline; pmc,clinical characteristics and diagnostic challe...,10.1016/j.jfma.2020.04.007,pmc7161491,32307322.0,els-covid,background/purpose current studies on pediatri...,2020-04-16,...,NaN,document_parses/pdf_json/44eec5c0f84d706903343...,document_parses/pmc_json/pmc7161491.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32307322/;...,215787843.0,157,journal of the formosan medical association,"['literature review', 'methodologies or experi...",4,background/purpose current studies on pediatri...


In [100]:
corpus_df = corpus_df[corpus_df["summarised_abstracts"].notnull()]
corpus_df = corpus_df[corpus_df["summarised_abstracts"].str.strip() != ""]

In [101]:
corpus_df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,referenced_by_count,JournalName_DOI,tags,TagCount,summarised_abstracts
0,iua8c4hy,300ff0def92740f7e1142036f9e6ae30845e13d2,medrxiv,covid-19 outbreak in oman: model-driven impact...,10.1101/2020.04.02.20050666,NaN,NaN,medrxiv,motivated by the rapid spread of covid-19 all ...,2020-04-06,...,NaN,document_parses/pdf_json/300ff0def92740f7e1142...,NaN,https://doi.org/10.1101/2020.04.02.20050666,215782459.0,10,NaN,"['methodologies or experimental designs', 'the...",4,motivated by the rapid spread of covid-19 all ...
1,xph08fwv,9a3e8c974bc68c29b0273ee771553046d7f37930,medrxiv,automated and semi-automated contact tracing: ...,10.1101/2020.04.14.20063636,NaN,NaN,medrxiv,introduction traditional approaches to case-fi...,2020-04-17,...,NaN,document_parses/pdf_json/9a3e8c974bc68c29b0273...,NaN,http://medrxiv.org/cgi/content/short/2020.04.1...,216055053.0,2,NaN,"['literature review', 'methodologies or experi...",4,introduction traditional approaches to case-fi...
2,aosmo568,ca6949e3e039a0e4098644b0b2df90606ae3a7ee,medrxiv,efficacy of remdesivir versus placebo for the ...,10.1101/2020.04.09.20059196,NaN,NaN,medrxiv,background: in spite of the global containment...,2020-04-14,...,NaN,document_parses/pdf_json/ca6949e3e039a0e409864...,NaN,http://medrxiv.org/cgi/content/short/2020.04.0...,215782112.0,1,NaN,"['literature review', 'methodologies or experi...",4,background: in spite of the global containment...
3,ujomta30,d831dbf38025a44e0436fd11c52af4db9eb7c5b0,medrxiv,hydroxychloroquine versus covid-19: a rapid sy...,10.1101/2020.04.14.20065276,NaN,NaN,medrxiv,background: coronavirus disease 2019 (covid-19...,2020-04-20,...,NaN,document_parses/pdf_json/d831dbf38025a44e0436f...,NaN,http://medrxiv.org/cgi/content/short/2020.04.1...,216035656.0,15,NaN,"['literature review', 'methodologies or experi...",4,background: coronavirus disease 2019 (covid-19...
4,xwlzq3m3,44eec5c0f84d7069033431dcb5a6adeb01b8ee0c,elsevier; medline; pmc,clinical characteristics and diagnostic challe...,10.1016/j.jfma.2020.04.007,pmc7161491,32307322.0,els-covid,background/purpose current studies on pediatri...,2020-04-16,...,NaN,document_parses/pdf_json/44eec5c0f84d706903343...,document_parses/pmc_json/pmc7161491.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32307322/;...,215787843.0,157,journal of the formosan medical association,"['literature review', 'methodologies or experi...",4,background/purpose current studies on pediatri...


In [102]:
corpus_df['combined_sum_text'] = corpus_df['title'] + " [SEP] " + corpus_df['summarised_abstracts']

In [105]:
len(corpus_df)

9028

## **Load our pre-trained model from HuggingFace**

In [106]:
model = SentenceTransformer("StephKeddy/sbert-IR-covid-search-v2")

In [107]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


## **Create SBERT embeddings**

In [108]:
# Extract the text to encode
texts_to_encode = corpus_df["combined_sum_text"].tolist()
doc_ids = corpus_df["cord_uid"].tolist()

# Encode with your model (normalised embeddings)
embeddings = model.encode(
    texts_to_encode,
    convert_to_tensor=False,
    normalize_embeddings=False,
    batch_size=32,
    show_progress_bar=True
)

Batches:   0%|          | 0/283 [00:00<?, ?it/s]

## **Create index of corpus embeddings**

In [109]:
# Convert to DataFrame
embedding_df = pd.DataFrame(embeddings, index=doc_ids)

# Rename columns to integers 0 to 767
embedding_df.columns = list(range(embedding_df.shape[1]))

# Ensure cord_uid is index
embedding_df.index.name = "cord_uid"

In [110]:
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
cord_uid,,,,,,,,,,,,,,,,,,,,,
iua8c4hy,-0.001748,-0.150236,-0.022601,-0.053634,0.021746,-0.029955,0.019537,0.026627,-0.000604,0.030359,...,-0.056845,-0.038077,0.004868,-0.046042,-0.043801,0.007834,0.050613,-0.090653,-0.049847,-0.016946
xph08fwv,-0.001371,-0.150174,-0.022172,-0.054260,0.021743,-0.029741,0.019361,0.026650,-0.000420,0.030700,...,-0.056485,-0.036888,0.004918,-0.045992,-0.043923,0.007977,0.050494,-0.090609,-0.049678,-0.017119
aosmo568,-0.001210,-0.150208,-0.023100,-0.054301,0.022039,-0.029163,0.018678,0.026579,-0.001255,0.031464,...,-0.055740,-0.036123,0.005413,-0.045411,-0.044023,0.008077,0.050697,-0.090378,-0.049438,-0.017414
ujomta30,-0.001820,-0.150561,-0.022838,-0.054073,0.021424,-0.029502,0.018979,0.026613,-0.000882,0.030880,...,-0.056492,-0.037084,0.004968,-0.045862,-0.044074,0.007679,0.050970,-0.090996,-0.049502,-0.017325
xwlzq3m3,-0.002497,-0.148818,-0.023032,-0.053294,0.021572,-0.029733,0.019045,0.026848,-0.000380,0.031488,...,-0.056274,-0.036499,0.005378,-0.045763,-0.044336,0.007590,0.050223,-0.091120,-0.048848,-0.017314


In [111]:
len(embedding_df)

9028

## **Export indexed embeddings**

In [112]:
half = len(embedding_df) // 2

In [113]:
embedding_part1 = embedding_df.iloc[:half]
embedding_part2 = embedding_df.iloc[half:]

In [115]:
embedding_part1.to_csv("embeddings_part1.csv.gz", compression="gzip")
embedding_part2.to_csv("embeddings_part2.csv.gz", compression="gzip")

## **Export corresponding metadata for embeddings**

In [116]:
corpus_part1 = corpus_df.iloc[:half]
corpus_part2 = corpus_df.iloc[half:]

In [117]:
corpus_part1.to_csv("metadata_part1_final.csv", index=False)
corpus_part2.to_csv("metadata_part2_final.csv", index=False)

## **Retrieval & Ranking**
Document retrieval is performed using cosine similarity on vector embeddings. These embeddings are stored in a compressed CSV file (.csv.gz) hosted on GitHub. At runtime, the file is downloaded, decompressed, and loaded into memory for efficient similarity search through streamlit.

The below is a test run of the retrieval process using sklearn and the combined test/validation queries that weren't used to train the SBERT model

In [118]:
queries_url= "https://raw.githubusercontent.com/Ron-DS-AI/Information_Retrieval/refs/heads/main/DEMO_test_queries.csv"

In [119]:
queries_df = pd.read_csv(queries_url)

In [120]:
def search(query, corpus_df, model, top_k=50):
    # Encode query
    query_embedding = model.encode(query, convert_to_tensor=False, normalize_embeddings=True).reshape(1, -1)

    # Get document embeddings from corpus
    embedding_cols = list(range(768))
    doc_embeddings = corpus_df[embedding_cols].values

    # Compute cosine similarities
    similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Get top-k most similar documents
    top_indices = similarities.argsort()[-top_k:][::-1]

    # Collect and return results as DataFrame
    top_results = corpus_df.iloc[top_indices].copy()
    top_results["similarity"] = similarities[top_indices]
    top_results = top_results.reset_index(drop=False)  # ensure cord_uid is present

    # Reorder + rename columns for clarity
    return top_results[[
        "cord_uid", "similarity", "title", "abstract", "publish_time"
    ]]

In [121]:
final_index_df = corpus_df.merge(
    embedding_df,
    left_on="cord_uid",
    right_index=True,
    how="inner"
)

In [122]:
shortform_results_df = search("coronavirus and ACE inhibitors", final_index_df, model, top_k=50)
shortform_results_df.head()

,cord_uid,similarity,title,abstract,publish_time
0,jr8d6lud,0.999909,covid-19 and the renin-angiotensin system,covid-19 and the renin-angiotensin system,2020-03-27
1,kf2ddpo1,0.999907,covid-19 in patients with cardiovascular diseases,covid-19 in patients with cardiovascular diseases,2020-03-31
2,mv6zw1gs,0.999906,estimating case fatality rates of covid-19,estimating case fatality rates of covid-19,2020-03-31
3,4eo2t7b6,0.999906,estimating case fatality rates of covid-19,estimating case fatality rates of covid-19,2020-03-31
4,jnp2ldcj,0.999903,angiotensin receptor blockers and covid-19,angiotensin receptor blockers (arbs) exhibit m...,2020-06-30


During this process, we became aware that the cosine scores across all queries are incredibly high. The original data splits were thoroughly checked to ensure no data leakage. We then began testing on completely unrelated queries to determine if the model was working as expected; Whilst the cosine scores for these queires was stil high, there was a notable reduction, indicating the model is performing as expected.

As a result, the high cosine scores suggest that the SBERT embeddings are becoming over compressed due to the interanal normalisation within the underlying SBERT model. As a way to handle this, we have recalibrated the scores by creating a new min threshold.

## **Relevance score recalibration**
Recalibration is needed to transform tightly clustered cosine similarity scores into a more interpretable 0–100% relevance scale, enabling clearer ranking and user understanding

In [123]:
def recalibrate_score(score, low=0.999, high=1.0):
    """Rescales cosine similarity to a 0-100 relevance score."""
    score = np.clip(score, low, high)
    return round(100 * (score - low) / (high - low), 1)

In [124]:
# Apply recalibration to your search results
shortform_results_df["relevance_score"] = shortform_results_df["similarity"].apply(recalibrate_score)

In [125]:
shortform_results_df[["cord_uid", "similarity", "relevance_score", "title"]].head()

,cord_uid,similarity,relevance_score,title
0,jr8d6lud,0.999909,90.9,covid-19 and the renin-angiotensin system
1,kf2ddpo1,0.999907,90.7,covid-19 in patients with cardiovascular diseases
2,mv6zw1gs,0.999906,90.6,estimating case fatality rates of covid-19
3,4eo2t7b6,0.999906,90.6,estimating case fatality rates of covid-19
4,jnp2ldcj,0.999903,90.3,angiotensin receptor blockers and covid-19


In [126]:
longform_results_df = search("seeking range of information related to the number and proportion of super spreaders, their patterns of behavior that lead to spread, and potential prevention strategies targeted specifically toward super spreaders", final_index_df, model, top_k=50)
longform_results_df.head()

,cord_uid,similarity,title,abstract,publish_time
0,rncrk9zd,0.999926,personal protective equipment recommendations ...,personal protective equipment recommendations ...,2020-04-21
1,j0nm444m,0.999925,effective containment explains sub-exponential...,the recent outbreak of covid-19 in mainland ch...,2020-02-20
2,cv0mkpzd,0.999924,an urgent need for global preparedness against...,an urgent need for global preparedness against...,2018-04-04
3,id3iu30t,0.999923,covid-19 drive through testing: an effective s...,covid-19 drive through testing: an effective s...,2020-04-17
4,ktc6sp3f,0.999921,behavioral considerations and impact on person...,behavioral considerations and impact on person...,2020-05-31


In [127]:
# Apply recalibration to your search results
longform_results_df["relevance_score"] = longform_results_df["similarity"].apply(recalibrate_score)

In [128]:
longform_results_df[["cord_uid", "similarity", "relevance_score", "title"]].head()

,cord_uid,similarity,relevance_score,title
0,rncrk9zd,0.999926,92.6,personal protective equipment recommendations ...
1,j0nm444m,0.999925,92.5,effective containment explains sub-exponential...
2,cv0mkpzd,0.999924,92.4,an urgent need for global preparedness against...
3,id3iu30t,0.999923,92.3,covid-19 drive through testing: an effective s...
4,ktc6sp3f,0.999921,92.1,behavioral considerations and impact on person...
